In [77]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, KFold, RandomizedSearchCV
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from lightgbm import LGBMClassifier
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder 

Import Data & Data Understanding

In [78]:
policies = pd.read_csv(r"C:\Ziyuan Sui\MSBA\Fall22\Travelers\data\policies.csv")

In [79]:
policies.columns

Index(['id', 'Quote_dt', 'discount', 'Home_policy_ind', 'zip', 'state_id',
       'county_name', 'Agent_cd', 'quoted_amt', 'Prior_carrier_grp',
       'credit_score', 'Cov_package_type', 'CAT_zone', 'policy_id',
       'number_drivers', 'num_loaned_veh', 'num_owned_veh', 'num_leased_veh',
       'total_number_veh', 'convert_ind', 'split', 'primary_parking'],
      dtype='object')

In [80]:
#check null values
policies.isna().sum()

id                       0
Quote_dt                 0
discount                 0
Home_policy_ind          0
zip                    472
state_id                 0
county_name              0
Agent_cd              5430
quoted_amt             112
Prior_carrier_grp     5000
credit_score           300
Cov_package_type       770
CAT_zone               250
policy_id                0
number_drivers           0
num_loaned_veh           0
num_owned_veh            0
num_leased_veh           0
total_number_veh         0
convert_ind          12291
split                    0
primary_parking          0
dtype: int64

In [81]:
drivers = pd.read_csv(r"C:\Ziyuan Sui\MSBA\Fall22\Travelers\data\drivers.csv")

In [82]:
veh_df = pd.read_csv(r"C:\Ziyuan Sui\MSBA\Fall22\Travelers\data\vehicles.csv")

Data Cleaning & Feature Engineering

In [83]:
drivers.loc[drivers['living_status'].isna(), 'living_status'] = 'own'
drivers.loc[drivers['safty_rating'].isna(), 'safty_rating'] = drivers['safty_rating'].mean()
drivers['high_education_ind'] = drivers['high_education_ind'].fillna(0)

In [84]:
ls = drivers.groupby(['policy_id','living_status']).size().unstack('living_status', fill_value=0)
policies = pd.merge(policies, ls, on='policy_id')

In [85]:
ge = drivers.groupby(['policy_id','gender']).size().unstack('gender', fill_value=0)
policies = pd.merge(policies, ge, on='policy_id')

In [86]:
hei = drivers.groupby(['policy_id','high_education_ind']).size().unstack('high_education_ind', fill_value=0)
hei['high_education_ind'] = hei.loc[:,1]/(hei.loc[:,0]+hei.loc[:,1])
policies = pd.merge(policies, hei['high_education_ind'], on='policy_id')

In [87]:
ad = drivers.groupby(['policy_id']).agg(maxage=('age','max'), mixage=('age','min'), 
                                          maxrate=('safty_rating','max'), minrate=('safty_rating','min'), )
policies= pd.merge(policies, ad, on='policy_id')

In [88]:
policies.rename(columns={'mixage':'minage'},inplace=True)

In [89]:
te = drivers.groupby(['policy_id']).agg({'age': 'mean', 'safty_rating': 'mean'})
policies = pd.merge(policies, te, on='policy_id')

In [90]:
cat = ['Quote_dt', 'discount', 'Home_policy_ind', 'zip', 'state_id',
       'county_name', 'Agent_cd', 'Prior_carrier_grp',
       'Cov_package_type', 'CAT_zone',
       'number_drivers', 'num_loaned_veh', 'num_owned_veh',
       'num_leased_veh', 'total_number_veh', 'convert_ind',
       'primary_parking']

df = policies.drop(['policy_id', 'split'], axis=1)
df[cat] = df[cat].astype('object')
## In order to plot a better figure
df[['zip', 'Agent_cd']] = df[['zip', 'Agent_cd']].astype('float64')

In [91]:
veh_df['age'] = veh_df['age'].fillna(veh_df['age'].mean())

In [92]:
policy_df = policies

In [93]:
model = veh_df['make_model'].values
set(map(lambda s: s[:s.index(':')-1], model))
veh_df['make_model'] = list(map(lambda s: s[:s.index(':')-1], model))
tm = veh_df.groupby(['policy_id', 'make_model']).size().unstack('make_model', fill_value=0)
policy_df = pd.merge(policy_df, tm[['BMW', 'MERCEDES-BENZ']], on='policy_id')

In [94]:
policy_df = pd.merge(policy_df, veh_df.groupby(['policy_id']).agg({'age': 'mean'}), on='policy_id')
policy_df = policy_df.rename(columns={"age_x": "age", "age_y": "ave_age"})
policy_df = policy_df.drop(['num_loaned_veh', 'num_owned_veh',
       'num_leased_veh', 'total_number_veh'], axis=1)
tn = pd.crosstab(veh_df['policy_id'], veh_df['ownership_type'])
tn['total_number_veh'] = tn.sum(axis=1)
policy_df = pd.merge(policy_df, tn, on='policy_id')

In [95]:
policy_df['year'] = policy_df['Quote_dt'].apply(lambda s: s[:4])
policy_df['month'] = policy_df['Quote_dt'].apply(lambda s: s[5:7])
policy_df['year_month'] = policy_df['year'].apply(str)+policy_df['month'].apply(str)

In [96]:
from datetime import datetime

def days_between(d1, d2):
    d1 = datetime.strptime(d1, "%Y-%m-%d")
    d2 = datetime.strptime(d2, "%Y-%m-%d")
    return abs((d2 - d1).days)

In [97]:
earlist = min(policy_df['Quote_dt'])
policy_df['Date'] = policy_df['Quote_dt'].apply(lambda s: days_between(s, earlist))

In [98]:
policy_df['state_county'] = policy_df['state_id']+policy_df['county_name']
policy_df['credit_score'] = policy_df['credit_score']-policy_df['credit_score'].min()
policy_df['Home_policy_ind'] = policy_df[['Home_policy_ind']].replace(['Y'], '1')
policy_df['Home_policy_ind'] = policy_df[['Home_policy_ind']].replace(['N'], '0')
policy_df['Home_policy_ind'] = policy_df['Home_policy_ind'].astype('int64')

In [99]:
policy_df['zip'] = policy_df['zip'].fillna(0).astype('int64')
policy_df['Agent_cd'] = policy_df['Agent_cd'].fillna(0).astype('int64')
policy_df['CAT_zone'] = policy_df['CAT_zone'].fillna(0)
policy_df['Cov_package_type'] = policy_df['Cov_package_type'].fillna('Medium')

In [100]:
drop_list = ['Quote_dt', 'state_id', 'county_name',  'year', 'month', 'primary_parking']
policy_df = policy_df.drop(drop_list, axis=1)

In [101]:
list1 = ['discount', 'Prior_carrier_grp', 'Cov_package_type', 'year_month', 
       'zip', 'Agent_cd', 'CAT_zone', 'state_county']

for col in list1:
    le = LabelEncoder()
    policy_df[col] = le.fit_transform(list(policy_df[col].astype(str).values))

In [102]:
policy_df.columns

Index(['id', 'discount', 'Home_policy_ind', 'zip', 'Agent_cd', 'quoted_amt',
       'Prior_carrier_grp', 'credit_score', 'Cov_package_type', 'CAT_zone',
       'policy_id', 'number_drivers', 'convert_ind', 'split', 'dependent',
       'own', 'rent', 'F', 'M', 'high_education_ind', 'maxage', 'minage',
       'maxrate', 'minrate', 'age', 'safty_rating', 'BMW', 'MERCEDES-BENZ',
       'ave_age', 'leased', 'loaned', 'owned', 'total_number_veh',
       'year_month', 'Date', 'state_county'],
      dtype='object')

In [103]:
drop_list2 = ['id']
policy_df = policy_df.drop(drop_list2, axis=1)

policy_df.to_csv(r"C:\Ziyuan Sui\MSBA\Fall22\Travelers\data\finalized.csv")